In [ ]:
from google.colab import files
uploded = files.upload()

Saving Survey Data_shipping.csv to Survey Data_shipping.csv


In [51]:
import pandas as pd
import re

In [52]:
#importing the dataset
df = pd.read_csv("Survey Data_shipping.csv", encoding='latin-1')

In [53]:
df.head(3)

,ï»¿OCF Journey,Response Ticker - Why you gave this number in the survey,Why - issue Category,Why - Keywords/Words related to issue,Why - # of Words,Response Ticker - What we could do better?,What - Issue,What - Words related to issue,What - # of Words,Application area,Comment_Metadata1,Comment_Metadata2,Q1 - How likely are you to recommend UPS to a friend or colleague?,Survey Date (+00:00 GMT),Unnamed: 14,Unnamed: 15
0,Generate a Shipment (Digital),Easy,Layout/Navigation/Usablity,NaN,1,Occasionally the driver will pick up boxes too...,NaN,NaN,27,NaN,Logged In,SinglePage,10,8/9/2022 18:21,NaN,NaN
1,Generate a Shipment (Digital),Easy,Layout/Navigation/Usablity,NaN,1,"DonÃ¢â¬â¢t use USPS, nothing gets here on ti...",NaN,NaN,17,NaN,Logged In,Guided,8,8/5/2021 16:52,NaN,Layout/Navigation/Usablity
2,Generate a Shipment (Digital),easy,Layout/Navigation/Usablity,NaN,1,Email our invoices instead of mailing them.,NaN,NaN,7,NaN,Logged In,SinglePage,9,2/24/2021 12:20,NaN,Technical/Performance


In [54]:
#extract important columns
review = df[['Response Ticker - Why you gave this number in the survey', 'Why - issue Category' ]]

In [55]:
review.head()

,Response Ticker - Why you gave this number in the survey,Why - issue Category
0,Easy,Layout/Navigation/Usablity
1,Easy,Layout/Navigation/Usablity
2,easy,Layout/Navigation/Usablity
3,easy,Layout/Navigation/Usablity
4,Simple,Layout/Navigation/Usablity


In [56]:
review['Why - issue Category'].value_counts()

Operational                   2147
Layout/Navigation/Usablity    1973
Others                         595
Pricing/Cost                   525
Support/Sales                  430
Technical/Performance          224
others                          52
Operational                      5
Name: Why - issue Category, dtype: int64

In [57]:
#replaceing the case sensetive and space problems of 'Operations ' to 'Operations' and 'others' to 'Others'
review['Why - issue Category']= review['Why - issue Category'].replace({'Operational ':'Operational', 'others': 'Others'})


C:\Users\User\AppData\Local\Temp\ipykernel_2960\1068644813.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review['Why - issue Category']= review['Why - issue Category'].replace({'Operational ':'Operational', 'others': 'Others'})


In [58]:
review['Why - issue Category'].value_counts()

Operational                   2152
Layout/Navigation/Usablity    1973
Others                         647
Pricing/Cost                   525
Support/Sales                  430
Technical/Performance          224
Name: Why - issue Category, dtype: int64

In [59]:
review.head()

,Response Ticker - Why you gave this number in the survey,Why - issue Category
0,Easy,Layout/Navigation/Usablity
1,Easy,Layout/Navigation/Usablity
2,easy,Layout/Navigation/Usablity
3,easy,Layout/Navigation/Usablity
4,Simple,Layout/Navigation/Usablity


In [60]:
#changing the column name 
review = review.rename(columns={'Response Ticker - Why you gave this number in the survey':'response', 'Why - Issue catagory ': 'category'})

In [61]:
review.head()

,response,Why - issue Category
0,Easy,Layout/Navigation/Usablity
1,Easy,Layout/Navigation/Usablity
2,easy,Layout/Navigation/Usablity
3,easy,Layout/Navigation/Usablity
4,Simple,Layout/Navigation/Usablity


In [62]:
# review['response']

In [63]:
#checking the drop values
review.isna().sum()

response                0
Why - issue Category    0
dtype: int64

In [64]:
import re
import gensim
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import nltk
from xgboost import XGBClassifier
import numpy as np

In [65]:
#loaidng the stop words from nltk library
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
#clean the non letters characters
def clean_lower(text):
    clean = re.sub('[^a-zA-Z]', ' ', str(text)).lower()
    return clean

In [67]:
#downloading the english stopwords 
stopwords_english = stopwords.words('english')

In [68]:
# stopwords_english

In [69]:
#stemmer from nltk library to extract the base words 
stemmer_english= nltk.SnowballStemmer('english')

In [70]:
stemmer_english.stem('walking')

'walk'

In [73]:
stopwords_1= stopwords.words('english')
#some more useless words from the texts 
more = ['u', 'im', 'c', 'o', 'hmm']
#adding the useless words to the stopwords 
stopwords_english =  stopwords_1 + more

In [189]:
#function for stemming of each sentences
def preprocess_data(text):
#     t = ''
    text = clean_lower(text)
    text = ' '.join(word for word in text.split(' ') if word not in stopwords_english)
    text =' '.join([(stemmer_english.stem(word)) for word in text.split(' ')])
    return text

In [110]:
# stopwords_1

In [190]:
preprocess_data('i loving data')

'love data'

In [178]:
np.array([1,2,3,4,5])

array([1, 2, 3, 4, 5])

In [193]:
#applying the stem function to the responses
review['response'] = review['response'].apply(lambda x: preprocess_data(x))

In [194]:
review['response'].head()

0     easi
1     easi
2     easi
3     easi
4    simpl
Name: response, dtype: object

In [195]:
review.head()

,response,Why - issue Category
0,easi,Layout/Navigation/Usablity
1,easi,Layout/Navigation/Usablity
2,easi,Layout/Navigation/Usablity
3,easi,Layout/Navigation/Usablity
4,simpl,Layout/Navigation/Usablity


In [196]:
#number of issue cateogry
review['Why - issue Category'].value_counts()

Operational                   2152
Layout/Navigation/Usablity    1973
Others                         647
Pricing/Cost                   525
Support/Sales                  430
Technical/Performance          224
Name: Why - issue Category, dtype: int64

In [197]:
#encoding the issue category as it is only 5
review['encoded_catagory'] = review['Why - issue Category'].factorize()[0]

In [198]:
review.head()

,response,Why - issue Category,encoded_catagory
0,easi,Layout/Navigation/Usablity,0
1,easi,Layout/Navigation/Usablity,0
2,easi,Layout/Navigation/Usablity,0
3,easi,Layout/Navigation/Usablity,0
4,simpl,Layout/Navigation/Usablity,0


In [199]:
# code, label = 
# review['Why - issue Category'].factorize()

In [106]:
# encoding = {}
# for inx, value in enumerate(label):
#   encoding[inx] = value
#   # print(inx, value
# print(encoding)


In [200]:
#selecting the dependant and independent variables 
X= review['response']
y = review['encoded_catagory']

In [201]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: encoded_catagory, dtype: int64

In [202]:
from sklearn.model_selection import train_test_split

In [203]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [204]:
print(X_train.shape[0], X_test.shape[0])
print(y_train.shape[0], y_test.shape[0])

4760 1191
4760 1191


In [205]:
#importing word2vec from gensim
import numpy as np
from gensim.models import Word2Vec

In [206]:
sentences= [review for review in X_train]

In [207]:
#word2vec (from gensim library) model for the sentences
model = Word2Vec(sentences, min_count=1, vector_size=500, workers=6, window=5, sg=0)

In [208]:
#creating function for embedding
def get_cbow_embeddings(model, corpus):
    embeddings = np.zeros((len(corpus), model.wv.vector_size))
    for i, sentence in enumerate(corpus):
        sentence_embeddings = [model.wv.get_vector(word) for word in sentence if model.wv.has_index_for(word)]
        if len(sentence_embeddings) > 0:
            embeddings[i] = np.mean(sentence_embeddings, axis=0)
    return embeddings

# Convert train and test sets to Word2Vec CBoW embeddings
X_train_w2v = get_cbow_embeddings(model, X_train)
X_test_w2v = get_cbow_embeddings(model, X_test)


In [209]:
X_train_w2v.shape

(4760, 500)

In [210]:
X_test_w2v.shape

(1191, 500)

In [211]:
#machine learnig model for classification 
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [212]:
gnb_w2v = GaussianNB()
gnb_w2v.fit(X_train_w2v, y_train)
y_pred_w2v = gnb_w2v.predict(X_test_w2v)

accuracy_w2v = accuracy_score(y_test, y_pred_w2v)
confusion_matrix_w2v = confusion_matrix(y_test, y_pred_w2v,labels=[1,0])
report_w2v = classification_report(y_test, y_pred_w2v)

In [213]:
print(report_w2v)

              precision    recall  f1-score   support

           0       0.50      0.14      0.22       398
           1       0.61      0.29      0.39       430
           2       0.26      0.32      0.29       143
           3       0.12      0.25      0.17       100
           4       0.06      0.51      0.11        41
           5       0.17      0.35      0.23        79

    accuracy                           0.25      1191
   macro avg       0.29      0.31      0.23      1191
weighted avg       0.44      0.25      0.28      1191



In [214]:
accuracy_w2v

0.2518891687657431

In [215]:
#tokenizing
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Using Countvectorization and Tfidf transformer

In [222]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()

In [225]:
X_train_vec = vec.fit_transform(X_train)
X_test_vec = vec.transform(X_test)

In [230]:
from sklearn.feature_extraction.text import TfidfTransformer

In [231]:
tfidf_transformer = TfidfTransformer().fit(X_train_vec)

In [232]:
X_train_vec_tfidf = tfidf_transformer.transform(X_train_vec)
X_test_vec_tfidf  = tfidf_transformer.transform(X_test_vec)

In [233]:
from sklearn.naive_bayes import MultinomialNB

In [221]:
X_train.head()

1705                                    keep chang websit
2549                                 web site user friend
295                                           good servic
4865    guy consist matter   long thing stay better good 
2285                                  alway seem consist 
Name: response, dtype: object

In [234]:
nb_model = MultinomialNB().fit(X_train_vec_tfidf, y_train)

In [235]:
y_predict = nb_model.predict(X_test_vec_tfidf)

In [ ]:
def preprocess_data(text):
    text = clean_lower(text)
    text = ' '.join(word for word in text.split(' ') if word not in stopwords_english)
    text = ' '.join(stemmer_english.stem(word) for word in text.split(' '))
    return text

In [237]:
from sklearn.pipeline import Pipeline

# MultinomialNB

In [239]:
#Pipeline for performing CountVectorization, tfidf, and classification algorithom fitting 
nlp_pipeline_mnb = Pipeline([
('bow', CountVectorizer(analyzer=preprocess_data)),
('tfidf', TfidfTransformer() ),
('classifer', MultinomialNB() )
])

In [240]:
nlp_pipeline_mnb.fit(X_train, y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function preprocess_data at 0x000001CE26F0DA20>)),
                ('tfidf', TfidfTransformer()), ('classifer', MultinomialNB())])

In [255]:
y_pred_mnb = nlp_pipeline_mnb.predict(X_test)

In [256]:
accuracy_score(y_pred_mnb, y_test)

0.5037783375314862

# SVC

In [238]:
def preprocess_data(text):
    text = clean_lower(text)
    text = ' '.join(word for word in text.split(' ') if word not in stopwords_english)
    text = ' '.join(stemmer_english.stem(word) for word in text.split(' '))
    return text

In [241]:
from sklearn.svm import SVC

nlp_pipeline_svc = Pipeline([
('bow', CountVectorizer(analyzer=preprocess_data)),
('tfidf', TfidfTransformer() ),
('classifer', SVC() )


])

In [242]:
nlp_pipeline_svc.fit(X_train, y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function preprocess_data at 0x000001CE26F0DA20>)),
                ('tfidf', TfidfTransformer()), ('classifer', SVC())])

In [243]:
y_predict_svc = nlp_pipeline_svc.predict(X_test)

In [244]:
accuracy_score(y_predict_svc, y_test)

0.6020151133501259

# XGBoost

In [280]:

nlp_pipeline_xgb = Pipeline([
('bow', CountVectorizer(analyzer=preprocess_data)),
('tfidf', TfidfTransformer() ),
('classifer', XGBClassifier() )
])

In [281]:
nlp_pipeline_xgb.fit(X_train, y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function preprocess_data at 0x000001CE26F0DA20>)),
                ('tfidf', TfidfTransformer()),
                ('classifer',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None...
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               objective='multi:softprob', predictor=None, ...))])

In [247]:
y_predict_xgb = nlp_pipeline_xgb.predict(X_test)

In [248]:
accuracy_score(y_predict_xgb, y_test)

0.5860621326616289

# Random Forest

In [249]:
from sklearn.ensemble import RandomForestClassifier

nlp_pipeline_rnf = Pipeline([
('bow', CountVectorizer(analyzer=preprocess_data)),
('tfidf', TfidfTransformer() ),
('classifer', RandomForestClassifier() )


])

In [250]:
nlp_pipeline_rnf.fit(X_train, y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function preprocess_data at 0x000001CE26F0DA20>)),
                ('tfidf', TfidfTransformer()),
                ('classifer', RandomForestClassifier())])

In [251]:
y_predict_rnf= nlp_pipeline_rnf.predict(X_test)

In [252]:
accuracy_score(y_test, y_predict_rnf)

0.6020151133501259

In [253]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predict_rnf))

              precision    recall  f1-score   support

           0       0.60      0.76      0.67       398
           1       0.59      0.75      0.66       430
           2       0.61      0.34      0.44       143
           3       0.76      0.19      0.30       100
           4       0.50      0.05      0.09        41
           5       0.64      0.27      0.38        79

    accuracy                           0.60      1191
   macro avg       0.62      0.39      0.42      1191
weighted avg       0.61      0.60      0.57      1191

